In [1]:
import json
import tensorflow as tf
# from keras.preprocessing import image
from keras.models import Sequential
from keras.utils import image_dataset_from_directory
from keras.layers import InputLayer, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.metrics import Accuracy, Precision, Recall
import cv2 as cv
from matplotlib import pyplot as plt

In [2]:
classes: list[str] = []

with open('land_use_images\\label_map.json', 'rb') as f:
    classes = json.loads(f.read())

classes = [x for x in classes.keys()]
(image_width, image_height) = 256, 256

In [3]:
'''
data_generator: image.ImageDataGenerator = image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
'''

"\ndata_generator: image.ImageDataGenerator = image.ImageDataGenerator(\n    rotation_range=40,\n    width_shift_range=0.2,\n    height_shift_range=0.2,\n    rescale=1./255,\n    shear_range=0.2,\n    zoom_range=0.2,\n    horizontal_flip=True,\n    fill_mode='nearest'\n)\n"

In [4]:
def normalizing_data(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

In [11]:
train_dataset: tf.data.Dataset = image_dataset_from_directory(
    "land_use_images\\images_train_test_val\\train",
    labels="inferred",
    label_mode="int",
    class_names=classes,
    color_mode="rgb",
    batch_size=50,
    image_size=(image_width, image_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True
)
train_dataset = train_dataset.map(normalizing_data)

Found 7350 files belonging to 21 classes.


In [13]:
test_dataset: tf.data.Dataset = image_dataset_from_directory(
    "land_use_images\\images_train_test_val\\test",
    labels="inferred",
    label_mode="int",
    class_names=classes,
    color_mode="rgb",
    batch_size=50,
    image_size=(image_width, image_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True
)
test_dataset = train_dataset.map(normalizing_data)

Found 1050 files belonging to 21 classes.


In [14]:
validation_dataset: tf.data.Dataset = image_dataset_from_directory(
    "land_use_images\\images_train_test_val\\validation",
    labels="inferred",
    label_mode="int",
    class_names=classes,
    color_mode="rgb",
    batch_size=50,
    image_size=(image_width, image_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True
)
validation_dataset = train_dataset.map(normalizing_data)

Found 2100 files belonging to 21 classes.


In [6]:
input_shape = (image_width, image_height, 3)

In [15]:
model = Sequential()

#model.add(InputLayer(shape=input_shape))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=21, activation='softmax'))  # Supondo 10 classes para a saída

#model.add(Conv2D(128, 3, activation='relu', padding='same'))
#model.add(Conv2D(128, 3, activation='relu', padding='same'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(258, 3, activation='relu', padding='same'))
#model.add(Conv2D(258, 3, activation='relu', padding='same'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Flatten())
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(rate=0.5))
#model.add(Dense(64, activation="relu")),
#model.add(Dropout(rate=0.5))
#model.add(Dense(21, activation='softmax'))

'''
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
'''



"\nmodel.add(Conv2D(32, (2, 2), input_shape=input_shape))\nmodel.add(Activation('relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n \nmodel.add(Conv2D(32, (2, 2)))\nmodel.add(Activation('relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n \nmodel.add(Conv2D(64, (2, 2)))\nmodel.add(Activation('relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n \nmodel.add(Flatten())\nmodel.add(Dense(64))\nmodel.add(Activation('relu'))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(1))\nmodel.add(Activation('sigmoid'))\n"

In [16]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

In [17]:
model.fit(train_dataset, epochs=5, steps_per_epoch=250, batch_size=32, validation_data=test_dataset)

Epoch 1/5
147/250 ━━━━━━━━━━━━━━━━━━━━ 2:13 1s/step - accuracy: 0.0681 - loss: 5.8134

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


250/250 ━━━━━━━━━━━━━━━━━━━━ 247s 965ms/step - accuracy: 0.0739 - loss: 4.9104 - val_accuracy: 0.0476 - val_loss: 3.8917
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 249s 991ms/step - accuracy: 0.1784 - loss: 2.6931 - val_accuracy: 0.0476 - val_loss: 4.5760
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 238s 948ms/step - accuracy: 0.3848 - loss: 2.0693 - val_accuracy: 0.0476 - val_loss: 5.6213
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 248s 987ms/step - accuracy: 0.6111 - loss: 1.3119 - val_accuracy: 0.0476 - val_loss: 6.7419
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.7972 - loss: 0.7332 - val_accuracy: 0.0476 - val_loss: 7.1665


In [18]:
model.evaluate(validation_dataset)

147/147 ━━━━━━━━━━━━━━━━━━━━ 59s 401ms/step - accuracy: 0.0515 - loss: 7.2375


[7.166549205780029, 0.0476190485060215]